# Scrape

In [ ]:
url = 'https://open.spotify.com/user/miro165sabo/playlist/2kxC9fJWg0aEanq3ylZR5i'

from requests_html import HTMLSession

session = HTMLSession()

In [ ]:
r = session.get(url)
r

In [ ]:
import json
songs = json.loads(r.html.find('script')[-5].text.split('Spotify.Entity =')[1].strip(' ;'))['tracks']['items']

In [ ]:
from pprint import pprint
#pprint(songs[0])

In [ ]:
data = []

for song in songs:
    track = song['track']
    
    info = {    
        'name': track['name'],
        'artists':[artist['name'] for artist in track['artists']],
        'release_date':track['album']['release_date']
    }
    
    info['artist_clean'] = info['artists'][0].split(',')[0]
    info['name_clean'] =  (info['name'].split('(')[0] if info['name'].endswith(')') else info['name']).split('-')[0].strip()
    #info['name_clean'] = info['name'].split('(')[0].split('-')[0].strip()
    data.append(info)
    
import pandas as pd  
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.options.display.width = 50
# pd.options.display.max_colwidth = 50
df = pd.DataFrame(data) 
df

# Timeline

### From spotify album release_date (very inaccurate)

In [ ]:
%matplotlib inline
pd.Series([int(rd[:4]) for rd in df['release_date']]).value_counts().sort_index().plot(kind='bar')

### From wikipedia (accurate) 

In [ ]:
import wikipedia


wikipedia.search("nicki french total eclipse of the heart")

# Download

In [ ]:
import subprocess
from pathlib import Path

downloaded = {p.stem.split('-')[-1] for p in Path('results/playlist/').glob("*.mp3")}

for _, song in df.iterrows():
    q = f"{song['artist_clean']} {song['name_clean']} hd"
    print(q)   
    
    try:
        r = session.get('https://www.youtube.com/results', params = {'search_query':q})
        vid = json.loads(r.html.find('script')[-3].text.split('=', 1)[1].split('; window')[0])['contents']['twoColumnSearchResultsRenderer']['primaryContents']['sectionListRenderer']['contents'][0]['itemSectionRenderer']['contents'][0]['videoRenderer']['videoId']             
        print(vid)
        if vid in downloaded: continue

        subprocess.run(f'youtube-dl -o "results/playlist/%(title)s-%(id)s.%(ext)s" --extract-audio --audio-format mp3 {vid}', shell=True)
        break
    except:
        print(f'Error: {q}')

In [ ]:
json.loads(r.html.find('script')[-3].text.split('=', 1)[1].split('; window')[0])['contents']['twoColumnSearchResultsRenderer']['primaryContents']['sectionListRenderer']['contents'][0]['itemSectionRenderer']['contents'][2]#

# Trash

In [ ]:
songs[0]['track']['name']

In [ ]:
songs[0]['track']['artists'][0]['name']

In [ ]:
songs[0]['track']['album']['release_date']